In [ ]:
!pip install tifffile

In [ ]:
import os
import zipfile
import tifffile as tiff
from PIL import Image, ImageFile

# Set PIL parameters to handle large images
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None

def cut_image_into_tiff_blocks(image_path, block_size):
    """
    Cut an image into TIFF blocks.

    Args:
        image_path (str): The path to the input TIFF image.
        block_size (tuple): A tuple containing the width and height of the blocks.

    Returns:
        list: A list containing the TIFF blocks as numpy arrays.
    """
    with tiff.TiffFile(image_path) as tif:
        image = tif.asarray()

        block_width, block_height = block_size
        total_height, total_width = image.shape[:2]

        num_horizontal_blocks = total_width // block_width
        num_vertical_blocks = total_height // block_height

        blocks = []

        for i in range(num_vertical_blocks):
            for j in range(num_horizontal_blocks):
                left = j * block_width
                top = i * block_height
                block = image[top:top+block_height, left:left+block_width]
                blocks.append(block)

        return blocks

# Path to the input image
image_path = '/content/2024-04-21-00_00_2024-04-21-23_59_Sentinel-2_L1C_Highlight_Optimized_Natural_Color.tiff'

# Define the size of the blocks
block_size = (120, 120)

# Cut the image into TIFF blocks
blocks = cut_image_into_tiff_blocks(image_path, block_size)

# Save blocks to a ZIP file
with zipfile.ZipFile('/content/blocks.zip', 'w') as zipf:
    for index, block in enumerate(blocks):
        block_file = f'/content/block_{index}.tiff'
        tiff.imwrite(block_file, block)
        zipf.write(block_file, os.path.relpath(block_file, '/content/'))

# Remove TIFF files
for file in os.listdir('/content/'):
    if file.endswith('.tiff'):
        os.remove(os.path.join('/content/', file))